<a href="https://colab.research.google.com/github/kwakseoyeon/1test/blob/master/building_a_db_query_chatbot_app_based_on_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

목표 : 자연어 입력해서 SQL문을 만든다음, 이를 실행시켜 그 결과를 자연어로 사용자에게 전달하는 시나리오.

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql",
    filename="Chinook_Sqlite.sql",
)

('Chinook_Sqlite.sql', <http.client.HTTPMessage at 0x79fbb0c31750>)

# 데이터베이스 구축

In [ ]:
import sqlite3

# 데이터베이스 연결 생성 (데이터베이스가 없으면 새로 만듭니다)
conn = sqlite3.connect('Chinook.db')

with open("Chinook_Sqlite.sql", 'r') as file:
    script = file.read()

# 스크립트 실행
conn.executescript(script)

conn.close()

# LLM 환경 설정

In [ ]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [ ]:
import os

In [ ]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

chat_model = ChatOpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# DB 연결

In [ ]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
# 인자가 없으면 에러가 납니다.
def get_schema(_):

    print("###")
    print(_)
    print("in get_schema")
    print("###")

    return db.get_table_info()

In [ ]:
get_schema(_)

###
('Chinook_Sqlite.sql', <http.client.HTTPMessage object at 0x79fbb0c31750>)
in get_schema
###


'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

# 자연어를 SQL문으로 변환

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below,
write a SQL query that would answer the user's question:

{schema}

Question: {question}
SQL Query:"""

# "SQL Query:"가 왜 필요할까요?
# 답변유도문 + 응답형식지정

prompt_template = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

sql_gen_chain = (
        RunnablePassthrough.assign(schema = get_schema) # 기존 dict에다가 schema 키와 값을 추가한다.
        | prompt_template
        | chat_model
        | StrOutputParser()
    )

In [ ]:
sql_gen_chain.invoke({"question": "How many employees are there?"})

###
{'question': 'How many employees are there?'}
in get_schema
###


'SELECT COUNT(*) as totalEmployees\nFROM Employee;'

In [ ]:
sql_gen_chain.invoke({"question": "총 고객의 수는?"})

###
{'question': '총 고객의 수는?'}
in get_schema
###


'SELECT COUNT(CustomerId) AS TotalCustomers\nFROM Customer;'

In [ ]:
sql_gen_chain.invoke({"question": "가장 매출이 높은 음반은?"})

###
{'question': '가장 매출이 높은 음반은?'}
in get_schema
###


'SELECT Album.Title, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalRevenue\nFROM Album\nINNER JOIN Track ON Album.AlbumId = Track.AlbumId\nINNER JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId\nGROUP BY Album.AlbumId\nORDER BY TotalRevenue DESC\nLIMIT 1;'

In [ ]:
sql_gen_execute_chain = (
        RunnablePassthrough.assign(schema = get_schema) # 기존 dict에다가 schema 키와 값을 추가한다.
        | prompt_template
        | chat_model
        | StrOutputParser()
        | db.run
    )

In [ ]:
sql_gen_execute_chain.invoke({"question": "총 고객의 수는?"})

###
{'question': '총 고객의 수는?'}
in get_schema
###


'[(59,)]'

# SQL문의 실행결과를 사용자에게 잘 전달하는 체인 구성

예를들어 실행결과 '8' 이렇게 나왔을때, 실행결과를 잘 설명하려면?
답변: 8입니다.

편지봉투
- 총 직원수는 몇명입니까?
- Table Schema
- SELECT COUNT(EmployeeId) AS TotalEmployees FROM Employee;
- 8

답변: 총 직원수는 8명입니다.

즉 필요한 변수는 총 4개입니다.

In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response in Korean:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""

prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.output_parser import StrOutputParser

'''
sql_response_chain = (
        RunnablePassthrough.assign(schema = get_schema) # 기존 dict에다가 schema 키와 값을 추가한다.
        | prompt_template
        | chat_model
        | StrOutputParser()
    )
'''

full_chain = (
    # {"question":"..."}
    RunnablePassthrough.assign(query = sql_gen_chain)
    # {"question":"...", "query":"..."} > x
    | RunnablePassthrough.assign(
        schema = get_schema,
        response = lambda x: db.run(x["query"]),
    )
    # {"question":"...", "query":"...", "schema":"...", "response":"..."}
    | prompt_response
    | chat_model
    | StrOutputParser()
)

In [ ]:
ans = full_chain.invoke({"question": "How many employees are there?"})

###
{'question': 'How many employees are there?'}
in get_schema
###
###
{'question': 'How many employees are there?', 'query': 'SELECT COUNT(EmployeeId) AS TotalEmployees\nFROM Employee;'}
in get_schema
###


In [ ]:
ans

'질문: 직원은 몇 명입니까?\nSQL 질의: SELECT COUNT(EmployeeId) AS TotalEmployees\nFROM Employee;\nSQL 응답: [(8,)] \n\n답변: 총 8명의 직원이 있습니다.'

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

full_chain = (
    RunnablePassthrough.assign(query = sql_gen_chain)
    # {"question":"...", "query":"..."} > x
    | RunnablePassthrough.assign(
        schema = get_schema,
        response = itemgetter("query") | RunnableLambda(db.run),
    )
    | prompt_response
    | chat_model
    | StrOutputParser()
)

In [ ]:
def llm(input_text):

    output = full_chain.invoke({"question": input_text})

    #print(output)

    return output

In [ ]:
def chat_with_user(user_message):
    ai_message = llm(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER > 매출이 가장 높은 직원은?
###
{'question': '매출이 가장 높은 직원은?'}
in get_schema
###
###
{'question': '매출이 가장 높은 직원은?', 'query': 'SELECT e.FirstName, e.LastName, SUM(il.UnitPrice * il.Quantity) AS TotalSales\nFROM Employee e\nJOIN Customer c ON e.EmployeeId = c.SupportRepId\nJOIN Invoice i ON c.CustomerId = i.CustomerId\nJOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId\nGROUP BY e.FirstName, e.LastName\nORDER BY TotalSales DESC\nLIMIT 1;'}
in get_schema
###
 A I > 제인 피콕이 가장 높은 매출을 기록했습니다. 833.04입니다.
USER > 가장 인기 없는 앨범은?>
###
{'question': '가장 인기 없는 앨범은?>'}
in get_schema
###
###
{'question': '가장 인기 없는 앨범은?>', 'query': 'SELECT a.Title AS AlbumTitle, COUNT(il.InvoiceId) AS SalesCount\nFROM Album a\nLEFT JOIN Track t ON a.AlbumId = t.AlbumId\nLEFT JOIN InvoiceLine il ON t.TrackId = il.TrackId\nGROUP BY a.AlbumId\nORDER BY SalesCount ASC\nLIMIT 1;'}
in get_schema
###
 A I > 가장 인기 없는 앨범은 "Battlestar Galactica: The Story So Far" 입니다.
USER > quit
